# direct exchange

直连exchange


In [1]:
import pika
import threading

TEST_EXCHANGE_NAME = "rb_test_direct_exchange"
connection_params = pika.ConnectionParameters(
    host="127.0.0.1",
    port=5672,
    virtual_host="/",
    credentials=pika.PlainCredentials("admin", "123456"),
)

# 生产者


In [2]:
rabbit_proudct_connection = pika.BlockingConnection(connection_params)
proudct_channel = rabbit_proudct_connection.channel()
proudct_channel.exchange_delete(TEST_EXCHANGE_NAME)
proudct_channel.exchange_declare(TEST_EXCHANGE_NAME, "direct")

# 声明队列，这里消费完了之后就直接删除
proudct_channel.queue_declare(queue="queue_1", auto_delete=True)
proudct_channel.queue_declare(queue="queue_2", auto_delete=True)
proudct_channel.queue_declare(queue="queue_3", auto_delete=True)

proudct_channel.queue_bind(queue="queue_1", exchange=TEST_EXCHANGE_NAME, routing_key="我是队列1")
proudct_channel.queue_bind(queue="queue_2", exchange=TEST_EXCHANGE_NAME, routing_key="我是队列2")
proudct_channel.queue_bind(queue="queue_3", exchange=TEST_EXCHANGE_NAME, routing_key="我是队列3")

# 发送消息，指定交换机、routing_key
proudct_channel.basic_publish(exchange=TEST_EXCHANGE_NAME, routing_key="我是队列1", body=b"kagura_mea1")

proudct_channel.basic_publish(exchange=TEST_EXCHANGE_NAME, routing_key="我是队列2", body=b"kagura_mea2")
proudct_channel.basic_publish(exchange=TEST_EXCHANGE_NAME, routing_key="我是队列2", body=b"kagura_mea3")

proudct_channel.basic_publish(exchange=TEST_EXCHANGE_NAME, routing_key="我是队列3", body=b"kagura_mea4")
proudct_channel.basic_publish(exchange=TEST_EXCHANGE_NAME, routing_key="我是队列3", body=b"kagura_mea5")
proudct_channel.basic_publish(exchange=TEST_EXCHANGE_NAME, routing_key="我是队列3", body=b"kagura_mea6")

proudct_channel.close()
rabbit_proudct_connection.close()

# 消费者


In [3]:
def on_message_callback(ch, method, properties, body):
    print(body.decode("utf-8"), f"channel id: {id(ch)}")
    ch.basic_ack(delivery_tag=method.delivery_tag)


def consume_message(queue_name: str, routing_key: str):
    consumer_connection = pika.BlockingConnection(connection_params)
    consumer_channel = consumer_connection.channel()
    consumer_channel.exchange_declare(TEST_EXCHANGE_NAME, "direct")
    consumer_channel.queue_declare(queue=queue_name, auto_delete=True)
    consumer_channel.queue_bind(queue_name, TEST_EXCHANGE_NAME, routing_key)
    consumer_channel.basic_consume(queue_name, on_message_callback=on_message_callback)
    consumer_channel.start_consuming()


t1 = threading.Thread(target=consume_message, args=("queue_1", "我是队列1")).start()
t2 = threading.Thread(target=consume_message, args=("queue_2", "我是队列2")).start()
t3 = threading.Thread(target=consume_message, args=("queue_3", "我是队列3")).start()

kagura_mea1 channel id: 2097868762624
kagura_mea2 channel id: 2097868757248
kagura_mea3 channel id: 2097868757248
kagura_mea4 channel id: 2097869385792
kagura_mea5 channel id: 2097869385792
kagura_mea6 channel id: 2097869385792
